### Dependencies

In [1]:
import sqlite3 # Database library.
import os # Folder management library.
import pickle # Serializing module.
import numpy as np # Scientific computing library.
import matplotlib.pyplot as plt # Plotting library.
from mpl_toolkits.mplot3d import Axes3D # 3D plotting tool.
from sklearn.neighbors import LocalOutlierFactor # Outlier Dection library.
from heapq import nsmallest # For finding closest period inlier to outlier

### Retreive Examples from Database

In [2]:
# Initializing database and cursor.
star_data_db = sqlite3.connect('star_data.db')
star_data_cursor = star_data_db.cursor()

# Retrieving star_data from database.
classes = ['cep_1o', 'cep_f', 'dsct','eb_ec', 'eb_ed', 'eb_esd', 'lpv_mira_agb_c', 'lpv_mira_agb_o', 'lpv_osarg_agb',
           'lpv_osarg_rgb', 'lpv_srv_agb_c', 'lpv_srv_agb_o', 'rrab', 'rrc', 'rrd', 'rre', 't2cep']

X, Y = [], []
for label, classv in enumerate(classes):
    temp_X, temp_Y = [], []
    star_data_cursor.execute('SELECT star_features FROM '+classv)
    for row in star_data_cursor.fetchall()[:600]:
        # Deserializing features.
        features = pickle.loads(row[0])
        temp_X.append(features)
        temp_Y.append([label])
    X.append(np.array(temp_X))
    Y.append(np.array(temp_Y))
    
names = []
for classv in classes:
    temp_names = []
    star_data_cursor.execute('SELECT star_name FROM '+classv)
    for row in star_data_cursor.fetchall()[:600]:
        temp_names.append(row[0])
    names.append(temp_names)
        

# Close cursor and database    
star_data_cursor.close
star_data_db.close()

print('done')

done


### Unsupervised Outlier Detection using Local Outlier Factor (LOF)
The LOF score of an observation is equal to the ratio of the average local density of its k-nearest neighbors, and its own local density: a normal instance is expected to have a local density similar to that of its neighbors, while abnormal data are expected to have much smaller local density. LOF can detect both local and global outliers.
<img src="img/lof.png", width=400, height=auto>


### 20 Best Outlier Candidates

In [3]:
# Constructing data.
names_t2cep = names[16]
X_t2cep = X[16]

best_outliers = []

# Recursively applying lof algorithm to obtain 20 best outliers.
for n in range(0,20):
    
    # Fitting the Local Outlier Detection model to the sample data set, and looking for 1 outlier. 
    lof = LocalOutlierFactor(n_neighbors=20, contamination=1/len(X_t2cep))
    outlier_pred = lof.fit_predict(X_t2cep)

    # Find the indices if all outliers.
    outlier_ind = np.where(outlier_pred == -1)[0]
    
    # Find the names of the outliers, and their coordinates.
    for ind in outlier_ind:
        best_outliers.append(names_t2cep[ind])
    
    # Remove outlier from data set.
    X_t2cep = np.delete(X_t2cep, outlier_ind[0], 0)
    names_t2cep = np.delete(names_t2cep, outlier_ind[0], 0)
    
print('The 10 best outliers are: ', best_outliers)



The 10 best outliers are:  ['OGLE-LMC-T2CEP-113.dat', 'OGLE-BLG-T2CEP-177.dat', 'OGLE-LMC-T2CEP-115.dat', 'OGLE-BLG-T2CEP-351.dat', 'OGLE-BLG-T2CEP-345.dat', 'OGLE-BLG-T2CEP-354.dat', 'OGLE-LMC-T2CEP-200.dat', 'OGLE-BLG-T2CEP-350.dat', 'OGLE-BLG-T2CEP-352.dat', 'OGLE-LMC-T2CEP-061.dat', 'OGLE-LMC-T2CEP-137.dat', 'OGLE-SMC-T2CEP-21.dat', 'OGLE-LMC-T2CEP-110.dat', 'OGLE-BLG-T2CEP-293.dat', 'OGLE-BLG-T2CEP-141.dat', 'OGLE-BLG-T2CEP-202.dat', 'OGLE-BLG-T2CEP-042.dat', 'OGLE-SMC-T2CEP-39.dat', 'OGLE-LMC-T2CEP-077.dat', 'OGLE-SMC-T2CEP-24.dat']


### Similar Period Inliers to Outliers

In [7]:
feature_names = ['Amplitude', 'AndersonDarling', 'Autocor_length', 'Beyond1Std', 'CAR_mean', 'CAR_sigma', 'CAR_tau', 
                 'Con', 'Eta_e', 'FluxPercentileRatioMid20', 'FluxPercentileRatioMid35', 'FluxPercentileRatioMid50',
                 'FluxPercentileRatioMid65','FluxPercentileRatioMid80', 'Freq1_harmonics_amplitude_0',
                 'Freq1_harmonics_amplitude_1', 'Freq1_harmonics_amplitude_2', 'Freq1_harmonics_amplitude_3',
                 'Freq1_harmonics_rel_phase_0', 'Freq1_harmonics_rel_phase_1', 'Freq1_harmonics_rel_phase_2',
                 'Freq1_harmonics_rel_phase_3', 'Freq2_harmonics_amplitude_0', 'Freq2_harmonics_amplitude_1', 
                 'Freq2_harmonics_amplitude_2', 'Freq2_harmonics_amplitude_3', 'Freq2_harmonics_rel_phase_0',
                 'Freq2_harmonics_rel_phase_1', 'Freq2_harmonics_rel_phase_2', 'Freq2_harmonics_rel_phase_3',
                 'Freq3_harmonics_amplitude_0', 'Freq3_harmonics_amplitude_1', 'Freq3_harmonics_amplitude_2', 
                 'Freq3_harmonics_amplitude_3', 'Freq3_harmonics_rel_phase_0', 'Freq3_harmonics_rel_phase_1',
                 'Freq3_harmonics_rel_phase_2', 'Freq3_harmonics_rel_phase_3', 'Gskew', 'LinearTrend', 'MaxSlope',
                 'Mean', 'Meanvariance', 'MedianAbsDev', 'MedianBRP', 'PairSlopeTrend', 'PercentAmplitude', 
                 'PercentDifferenceFluxPercentile', 'Period_fit', 'PeriodLS', 'Psi_CS', 'Psi_eta', 'Q31', 'Rcs',
                 'Skew', 'SlottedA_length', 'SmallKurtosis', 'Std', 'StetsonK', 'StetsonK_AC', 'StructureFunction_index_21',
                 'StructureFunction_index_31', 'StructureFunction_index_32', 'Colour']

t2cep_periods = X[16][:,feature_names.index('Period_fit')].tolist()
similar_inliers = []

for n in range(0,len(best_outliers)):
    
    ind_outlier = names[16].index(best_outliers[n])
    outlier_period = t2cep_periods[ind_outlier]

    # Find 10 inliers that have a similar period to outlier.
    inlier_periods = nsmallest(21, t2cep_periods, key=lambda x: abs(x-outlier_period))
    
    temp_names = []
    for inlier,period in enumerate(inlier_periods):
        ind_inlier = t2cep_periods.index(inlier_periods[inlier])
        inlier_name = names[16][ind_inlier]
        temp_names.append((inlier_name, period))
    
    similar_inliers.append(temp_names)
    
from pprint import pprint    
pprint(similar_inliers)

[[('OGLE-LMC-T2CEP-113.dat', 3.0853123206644857),
  ('OGLE-LMC-T2CEP-073.dat', 3.088140338248049),
  ('OGLE-BLG-T2CEP-052.dat', 3.0884557844690974),
  ('OGLE-BLG-T2CEP-316.dat', 3.126295818584072),
  ('OGLE-BLG-T2CEP-141.dat', 3.164786926994907),
  ('OGLE-BLG-T2CEP-099.dat', 2.9913462424347226),
  ('OGLE-BLG-T2CEP-152.dat', 3.1972930245854765),
  ('OGLE-SMC-T2CEP-09.dat', 2.9712227025171627),
  ('OGLE-BLG-T2CEP-115.dat', 3.200648819680078),
  ('OGLE-BLG-T2CEP-070.dat', 2.9358741708542717),
  ('OGLE-LMC-T2CEP-049.dat', 3.23516969893993),
  ('OGLE-BLG-T2CEP-235.dat', 2.8842401216994458),
  ('OGLE-BLG-T2CEP-100.dat', 2.8746305643994225),
  ('OGLE-BLG-T2CEP-286.dat', 2.841048275054375),
  ('OGLE-LMC-T2CEP-145.dat', 3.3374764722417427),
  ('OGLE-SMC-T2CEP-30.dat', 3.388736848279458),
  ('OGLE-BLG-T2CEP-238.dat', 2.770107767624021),
  ('OGLE-LMC-T2CEP-085.dat', 3.405327660386357),
  ('OGLE-BLG-T2CEP-202.dat', 2.75932598960026),
  ('OGLE-BLG-T2CEP-039.dat', 2.7554615561307023),
  ('OGLE-LMC-T

### Comparing Outlier Feature to Spread of the Data

In [5]:
# Mean and std  of all labels.
mean_features = []
std_features = []

for n in range(0, len(X)):
    mean = np.mean(X[n], axis=0).tolist()
    std = np.std(X[n], axis=0).tolist()
    
    mean_features.append(mean)
    std_features.append(std)
    
mean_features = np.array(mean_features)
std_features = np.array(std_features)

# The spread of the data.
upper_spread = mean_features + std_features
lower_spread = mean_features - std_features

for n in range(0, len(best_outliers)):
    print(best_outliers[n])
    
    ind_outlier = names[16].index(best_outliers[n])
    outlier_features = X[16][ind_outlier]
    
    outlier_upper = np.greater(outlier_features, upper_spread[16])
    outlier_lower = np.less(outlier_features, lower_spread[16])
    
    ind_upper = np.where(outlier_upper == True)[0]
    ind_lower = np.where(outlier_lower == True)[0]
    
    
    print('These features of the outlier are greater than mean+std.\nFeature, Outlier_Value, Mean+STD, Ratio')
    for ind in ind_upper:
        ratio = outlier_features[ind]/upper_spread[16][ind]
        #if ratio>2 or ratio<0:
        print(feature_names[ind], outlier_features[ind], upper_spread[16][ind], ratio)
        
    print(' ')
    
    print('These features of the outlier are less than mean-std.\nFeature, Outlier_Value, Mean+STD, Ratio')
    for ind in ind_lower:
        ratio = outlier_features[ind]/lower_spread[16][ind]
        #if ratio<0.1:
        print(feature_names[ind], outlier_features[ind], lower_spread[16][ind], ratio)
            
    print(' ')
    


OGLE-LMC-T2CEP-113.dat
These features of the outlier are greater than mean+std.
Feature, Outlier_Value, Mean+STD, Ratio
Eta_e 72179.1242711 4637.00725128 15.5658855723
Freq2_harmonics_rel_phase_3 1.62922757548 1.3620042609 1.19619858928
MaxSlope 75.5287009051 11.7553206964 6.42506511358
Mean 17.1415656109 16.8272145777 1.01868110921
MedianBRP 0.448868778281 0.328835603131 1.36502487567
Psi_eta 1.4645688218 0.524158696208 2.79413244958
Colour -0.675573278029 -0.699791457956 0.965392289873
 
These features of the outlier are less than mean-std.
Feature, Outlier_Value, Mean+STD, Ratio
Beyond1Std 0.326696832579 0.332197500685 0.983441572876
FluxPercentileRatioMid35 0.310526315789 0.33457035249 0.928134586578
FluxPercentileRatioMid50 0.436842105263 0.490744940249 0.890161200727
FluxPercentileRatioMid65 0.584210526316 0.655975888689 0.890597560656
FluxPercentileRatioMid80 0.778947368421 0.837929540204 0.929609628312
Freq1_harmonics_amplitude_0 0.0405460086032 0.0861231711214 0.470790939015
F

### Which label does the outlier belong to?
Find which label best describes an outliers features.  This is done by checking if the outlier features are within spread of a particular label.

In [6]:
outlier_labels = []
for n in range(0, len(best_outliers)):

    ind_outlier = names[16].index(best_outliers[n])
    outlier_features = X[16][ind_outlier]
    
    labels_temp = []
    for i in range(0, len(outlier_features)):
        column_temp = []
        for label in range(0, 17):
            if (outlier_features[i]<upper_spread[label][i]) and (outlier_features[i]>lower_spread[label][i]):
                column_temp.append(label)
        labels_temp.append(column_temp)
        
    outlier_labels.append(labels_temp)
    
# Flatten lists in list.
flat_outlier_labels = []
for n in range(0, len(outlier_labels)):
    flat_temp = [item for sublist in outlier_labels[n] for item in sublist]
    flat_outlier_labels.append(flat_temp)

# Most frequenct label for outlier.
from collections import Counter
for k in range(0, len(flat_outlier_labels)):
    print(best_outliers[k])
    count = Counter(flat_outlier_labels[k])
    print(count.most_common())
    
        

OGLE-LMC-T2CEP-113.dat
[(3, 50), (5, 47), (15, 47), (11, 44), (4, 42), (10, 42), (9, 39), (16, 38), (8, 36), (2, 34), (14, 34), (0, 33), (6, 32), (12, 30), (13, 30), (1, 25), (7, 24)]
OGLE-BLG-T2CEP-177.dat
[(6, 41), (7, 35), (10, 29), (3, 27), (11, 26), (5, 25), (12, 25), (16, 25), (2, 23), (14, 23), (15, 23), (4, 22), (8, 21), (13, 20), (0, 19), (9, 19), (1, 18)]
OGLE-LMC-T2CEP-115.dat
[(16, 46), (6, 37), (7, 36), (3, 31), (12, 31), (2, 30), (4, 30), (5, 30), (15, 30), (1, 26), (10, 25), (0, 24), (9, 24), (14, 22), (11, 21), (13, 21), (8, 20)]
OGLE-BLG-T2CEP-351.dat
[(6, 31), (10, 28), (2, 27), (4, 26), (5, 26), (7, 26), (16, 25), (3, 23), (15, 22), (12, 19), (8, 18), (9, 17), (11, 17), (1, 15), (0, 12), (14, 12), (13, 11)]
OGLE-BLG-T2CEP-345.dat
[(6, 44), (7, 29), (16, 27), (10, 25), (12, 25), (3, 24), (0, 21), (1, 20), (8, 20), (14, 20), (2, 19), (5, 19), (9, 19), (13, 19), (15, 19), (4, 17), (11, 17)]
OGLE-BLG-T2CEP-354.dat
[(6, 50), (7, 35), (10, 28), (16, 27), (12, 26), (3, 24),